In [1]:
import requests
import os
from dotenv import load_dotenv
load_dotenv("sjoerdAzure.env")  # Load environment variables from .env file
import time

import typing

from sklearn.metrics import cohen_kappa_score, classification_report
import krippendorff
import yaml

import pandas as pd

import config
import src
import tqdm
import json
import numpy as np
import logging

#import cltrier_lib as lib

import yaml
pd.set_option('display.max_colwidth', 100) 


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\sstolwi\Git

In [3]:
#set up helper variables and functions:
CFG = config.Config()

def load_json(path: str):
    with open(path, encoding='utf-8') as fp:
        return json.load(fp)
    
#set option variables:

#set options to low temperature (0,1):
options_low_str = """
seed: 42
temperature: 0.1
"""

options_low = yaml.safe_load(options_low_str)

MODELsmall: str = 'llama3.1:8b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
MODELlarge: str = 'llama3.1:70b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
MODELgpt4o = "nf-gpt-4o-2024-08-06" # in principe is er nu van elk model een nf (no filter) en een normale versie beschikbaar, de no filter versies zijn alleen voor onderzoekers beschikbaar voor analyze van content die niet door de filter heen zou komen.
MODELgpt4T = "nf-gpt-4-turbo" # Can be gpt-35-turbo, gpt-4-turbo, gpt-4 or Meta-Llama-3-8B-Instruct.

options_zero_str = """
seed: 42
temperature: 0
"""
options_zero = yaml.safe_load(options_zero_str)

temperature_0 : int = 0
SEED: int = 42
MAX10: int = 10
TOPP1: int = 1


options_large_str = """
seed: 42
temperature: 0
num_predict: 2000
"""
options_large = yaml.safe_load(options_large_str)

#load environment variables:
api_key = os.environ.get('sjoerd_key')

#setttings:
api_endpoint = "https://ai-research-proxy.azurewebsites.net/chat/completions"
####### API REQUEST FORMATTING ######
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + api_key
}
 

In [13]:
# api_endpoint = "https://aichat-proxy-ryaod2azslas4.azurewebsites.net/chat/completions"
api_endpoint = "https://ai-research-proxy.azurewebsites.net/chat/completions"
 
######## TO FILL BY STUDENT #########
student_ai_name = "your AI"
student_api_key = api_key 
student_llm = "gpt-4o-2024-08-06" # Can be gpt-35-turbo, gpt-4-turbo, gpt-4 or Meta-Llama-3-8B-Instruct.
student_initial_prompt = "You are an assistant and students are going to ask you questions. You initiate the following conversation by asking the student what help they need."
#####################################
 
 
####### API REQUEST FORMATTING ######
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + student_api_key
}
 
data = {
    "model": student_llm,
    "messages": [],
}
######################################
 
####### MAIN FUNCTIONS ###############
def get_ai_answer(user_input):
    """Sends an API request to the UvA AI chatbot using the globally declared request data and updating it with user_input.
 
    Args:
        user_input (string): user request to the chatbot
 
    Returns:
        string: the chatbot's answer, taken from the API request response
    """
    data["messages"].append({
        "role": "user",
        "content": user_input
    })
 
    response = requests.post(url=api_endpoint, headers=headers, json=data)
 
    if response.status_code == 200:
        data_response = response.json()
        return(data_response["choices"][0]["message"]["content"])
    else:
        print(f"Failed to connect to API. Status code: {response.status_code}")
        print(response.text)
        print(response)
        return ""
 
####### MAIN LOOP ####################
print(f"Loading {student_ai_name} ...")
get_ai_answer(student_initial_prompt) # This gives the chatbot the initial prompt
 
user_input = input("[Type 'stop' to stop the conversation]\n<<You>> ")
 
while (user_input != "stop"):
    print(get_ai_answer(user_input))
    user_input = input("\n<<You>> ")

 




Loading your AI ...
Failed to connect to API. Status code: 401
{"error":{"message":"Authentication Error, All connection attempts failed","type":"auth_error","param":"None","code":"401"}}
<Response [401]>


KeyboardInterrupt: Interrupted by user

In [4]:
data["messages"].append({
        "role": "user",
        "content": 'hello how are you?'
    })
response = requests.post(url=api_endpoint, headers=headers, json=data)

In [5]:
response.status_code

200

In [7]:
response.json()

{'id': 'chatcmpl-AZywPot4QWGuUYoaBIUcxMAEKuYK2',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': "Hello! I'm here and ready to help. How are you doing today?",
    'role': 'assistant',
    'tool_calls': None,
    'function_call': None}}],
 'created': 1733139101,
 'model': 'gpt-4o-2024-08-06',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_04751d0b65',
 'usage': {'prompt_tokens': 12, 'completion_tokens': 15, 'total_tokens': 27},
 'service_tier': None}

In [11]:
response=response.json()
response["choices"][0]["message"]["content"]

"I'm here if you need anything! Let me know how I can assist you."

In [ ]:
response = requests.post(url=api_endpoint, headers=headers, json=data)

In [4]:
SYSTEM_claim: str = \
    """
        Instruction:

        You are a text annotation assistant. Analyze a collection of social media comments enclosed in chevrons <..>. Identify and list the claims within these comments. Claims can be related to events, issues, opinions or concerns in relation to the specified topic.
        Claims are defined as the main assertion or conclusion of an argument.
        You summarize each claim into a short simple sentence.

        Response format:

        You provide only the list of claims, separated by commas, without any additional text or explanations. If no claims can be identified, return an empty list [].

        Response format template:
        
        ["claim 1", "claim 2", ... "claim x"]
	"""

SYSTEM_expansion: str = \
    """
# Instruction\n\nRephrase a social media post to reflect its meaning within the context of a conversation thread:\n\n1. You'll receive a thread in chevrons `<...>` and a target post in double chevrons `<<...>>`.\n2. If the post is clear without context, repeat it as-is.\n3. If the post depends on context, expand it to include necessary details.\n4. Respond with only the expanded post. \n5. If the post does not refer to context provided in the thread, or if in doubt, respond with the exactly repeating the target post as you received it. If no target post was presented reply with an empty list [] \n\n## Example\n\n**Input**:\n- Thread: <'Comment 1', 'Comment 2', 'Comment 3'>\n- Target reply: `<<This is so relatable!>>`\n\n**Output**:\n- '[context from previous comments] is so relatable!'\n\n**Text:
    """

SYSTEM_claim_new: str = \
    """
# Instruction

Identify whether a claim adds new information (2), expands on information (1) or does not add new information (0) with respect to preceding claims. New information is present (2) if the claim introduces a new argument or perspective. 
A claim expands on an existing argument or perspective (1) if it strongly related to it, or it provides additional context or examples to an argument or perspective in a preceding claim, or if it provides a sub-argument (parent-child relation). 
If the claim is very similar to a preceding claim or does not add any perspective or argument, assign a value of 0. Follow these steps:


Follow these steps:

1. You will receive a target claim in double chevrons <<...>> along with the preceding claims in single chevrons <...>.
2. Identify if new information is presented in the target claim with respect to the preceding posts. 
4. If in doubt, assign a value of 0.

Respond with only the predicted class (0 or 1 or 2) of the request. Do not include any additional text or explanations.
Class:
	"""

SYSTEM_claim_simscore: str = \
    """
# Instruction

Identify degree of similarity of a claim with respect to preceding claims on an intiger scale of 1 (completely dissimilar) to 10 (identical). A score of 5 indicates that the claims share context or topic, but are otherwise different. 
Follow these steps:

1. You will receive a target claim in double chevrons <<...>> along with a JSON containing the preceding claims and their claim_index enclosed in single chevrons <...>.
2. Determine the most similar claim to the target claim in the preceding claims. If only one preceding claim is provided pick this claim as the most similar claim.
3. Find the claim_index of this most similar claim. If you can't decide which claim is most similar, return an empty list [].
4. Identify the degree of similarity of the target claim with respect to that claim on a range of 1-10. 
5. If you can't give a similarity score return an empty list []. If no target claim is provided or only '[]' return an empty list [] for both the most_similar_claim_index and the similarity_score.
6. Only respond with the claim_index and similarity score.

Response format in JSON:

[
    {
        "most_similar_claim_index": "1",
        "similarity_score": "1"	
    }
]
	"""

In [20]:
#load data:
dataset_claim_embeds = pd.read_parquet(f'{CFG.report_dir}/pubsphere.claim_embed.parquet')

In [21]:
#find threads with most political posts:
#select the rows in the dataset that are in a thread with more political posts than cutoff:   
# Define the cutoff value
cutoff_value = 8

# Group by 'thread_title' and count the number of political posts
thread_counts = dataset_claim_embeds.groupby('videoTitle')['Llama31_political_fill_8b_score'].sum()

# Filter the threads based on the cutoff value
pol_threads = thread_counts[thread_counts > cutoff_value].index

# Select the posts in the filtered threads
pol_posts = dataset_claim_embeds[dataset_claim_embeds['videoTitle'].isin(pol_threads)]

# Display the filtered posts
print(pol_posts.loc[pol_posts['Llama31_political_fill_8b_score'] == 1, 'commentText'].to_list())

["That's a vicious insult!!! What did a box of rocks ever do to you that you would slander it like that? I represent the coalition for mineral rights. Minerals have rights to.", 'I agree.  Maybe he was sending a message to his mistress.', 'President Trump made 8,158 false or misleading claims in his first two years ... besides his entire life, businesses, deals, have been a lie as well  .... should he apologize?', 'NPC-90210-RockTheVote   Heâ€™s getting his oil checked!', "Gee, someone is upset that they are trying to give Fox a run for their money.  Don't worry Sierra, Fox still controls that niche.", "What are they being false about?  It's a fact that the military assets are being moved.", 'Bernie is the most consistent and most  trustworthy progressive. Feel the Bern!', "USA Freedom Patriots made the Internet. \xa0Why don't you go back to fucking donkeys.", "What I seriously don't get and leaves me shaking my head in frustration:\n the USA doesn't want Iran to have nuclear weapons -

In [25]:
pol_posts.columns


Index(['StartDate', 'RecordedDate', 'IPAddress', 'Finished', 'Coder', 'ID',
       'Mark_ID', 'Genre', 'topiccode', 'Platform',
       ...
       'claim_optlow', 'claim_optlow_MXBAI', 'tfidf_embed_post',
       'embed_MXBAI_post', 'cosine_similarity_post_claim_MXBAI',
       'cosine_low_high_MXBAI', 'tfidf_embed_post_svd',
       'Llama31_political_post_8b', 'Llama31_political_fill_8b',
       'Llama31_political_fill_8b_score'],
      dtype='object', length=106)

In [47]:
test = requests.post(
    url=api_endpoint,
    headers=headers,
    json={
        'model': MODELgpt4o,
        'messages': [
            {
                "role": "system",
                "content": SYSTEM_expansion
            },
            {
                "role": "user",
                "content": f'"Thread":\n<{pol_posts["commentText"][:5].to_list()}>, "Target reply":<<{pol_posts["commentText"][5:6].to_list()}>>'
            }
        ],
        'temperature': temperature_0,  # Replace with your desired temperature value
        'seed': SEED  # Replace with your desired seed value
    }
)

print(f"Status code: {test.status_code}")
print(test.text)
print(test)

Status code: 200
{"id":"chatcmpl-AZzd5Nb7bD2no4qWQW3w0XWTqahOa","choices":[{"finish_reason":"stop","index":0,"message":{"content":"[\"Gee, someone is upset that they are trying to give Fox a run for their money.  Don't worry Sierra, Fox still controls that niche.\"]","role":"assistant","tool_calls":null,"function_call":null}}],"created":1733141747,"model":"gpt-4o-2024-08-06","object":"chat.completion","system_fingerprint":"fp_04751d0b65","usage":{"prompt_tokens":375,"completion_tokens":31,"total_tokens":406},"service_tier":null}
<Response [200]>


In [48]:
test=test.json()
test["choices"][0]["message"]["content"]

'["Gee, someone is upset that they are trying to give Fox a run for their money.  Don\'t worry Sierra, Fox still controls that niche."]'

In [33]:
pol_posts["commentText"][5:6].to_list()

["Gee, someone is upset that they are trying to give Fox a run for their money.  Don't worry Sierra, Fox still controls that niche."]

In [16]:
####now try to test argument quality based on rationality prompt using the DeliberativePolitics dataset by Jaidka et al. (2022)####
# see https://github.com/kj2013/deliberative-politics for more information on the dataset

#this is the data on which at least 3 out of 4 annotators agreed on the coding leading to 5585 out of 6000 posts.

#these were the instructions to the Mtruk workers:
#Short Instructions  
# This tweet is a reply on Twitter (i.e., a Tweet) to a United States member of the Congress. Please classify this tweet according to whether it (a) is about politics (b) is positive/respectful (c) uncivil (d) has a genuine question (e) has a justification (f) is constructive. Each HIT takes about 30 seconds. 
# Steps:  
# • Read the tweet. 
# • Determine which categories best describe the tweet. 
# Relevance  
# • YES: Whether this tweet is probably about politics, or 
# • NO: this tweet is irrelevant to politics. 
# Positive/Respectful  
# • YES: Whether this tweet shows respect or empathy towards others, or 
# • NO: This tweet is not particularly positive or respectful. 
# Uncivil  
# • YES: Abuses and sledging: Whether this tweet uses ideological extremes like “liberal potheads', abuses like 'ass' or 'moron', stereotypes like 'faggot' or 'backward' or 'terrorist' 
# • YES: Threatening: Whether this tweet threatens individual freedoms ('You people better shut up'), threatens someone or threatens democracy ('American people must take him down')  
# • YES: Exaggeration: Whether this tweet uses exaggerated arguments (e.g. “It’s very easy to solve all of this just keep your legs closed if you don’t want a baby.'), or 
# • NO: This tweet is not particularly uncivil. 
# Reciprocity  
# • YES: Whether this tweet asks questions that were designed to elicit opinions or information (Where is the money coming from? Increased taxes?'), or 
# • NO: This tweet does not ask a genuine question or asks rhetorical questions (“You have no idea how limiting Medicaid coverage can be, do you?'). 
# Justification  
# • YES: Personal: Whether this tweet contains personal feelings or experiences, or 
# • YES: Fact-based: Whether this tweet contains facts, links or evidence from other sources, or item NO: This tweet does not offer a justification.  
# Constructiveness  
# • YES: Fact-checking: Whether this tweet contains fact-checking '(1) that’s not a real quote 2) more importantly, since then the DNC has embraced racially progressive stances... Mostly.') (“Not exactly true...she’s tried to invent a Native American heritage that failed epically') 
# • YES: Common ground: Whether this tweet contains a search for common ground (“You are undoubtedly right (correct, too). No matter how Conservative I am I am still a Mom and my heart strings get tugged easily.') (“I’m all for progressive change but too much will lead to repeat 2016') (“We can keep getting lost in the weeds') (“We are not all like that :)') 
# • YES: Solution: Whether this tweet contains a solution (“It would be WONDERFUL if the House & Senate committees looked into..')(“Also, no one is blaming Pence, Sec. Price for not getting it passed. Why not?') 
# • NO: This tweet is not constructive.
#  
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

#load data:
data_jaidka: pd.DataFrame = pd.read_csv('data/jaidka2022/TwitterDeliberativePolitics.csv')

In [73]:
data_jaidka

,message_id,message,Constructiveness,Justification,Justification_internal,Justification_external,Relevance,Reciprocity,Empathy_Respect,Uncivil_abuse
0,1,@USER- #GrahamCassidy will devastate #MilitaryFamilies w/ kids like Justin who need #Medicaid. P...,0,1,0,0,1.0,1.0,1.0,1.0
1,2,@USER- The US people &amp; Minnesotans must see the Senate Ethics investigation committee hearin...,0,1,0,0,1.0,1.0,1.0,1.0
2,4,"=@USER - ""we all want the same thing when you look at the big picture""",0,1,0,0,1.0,NaN,1.0,NaN
3,5,@USER - A poison in our island - Rising seas caused by climate change are seeping inside a Unite...,0,1,0,0,1.0,0.0,NaN,0.0
4,6,=@USER - hypocrite. You are A porn surfer and claim to be holier than thou.,0,0,0,0,NaN,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
5580,5984,@USER/please stand up for our democracy!,0,0,0,0,1.0,0.0,1.0,0.0
5581,5986,"@USER: $3,124,273 from the NRA during her career",0,1,0,0,1.0,0.0,NaN,0.0
5582,5987,"@USER: Glad you bribed well to cheat, lie, scheme. You're name is mud. @USER (who did not suppor...",0,0,0,0,1.0,0.0,0.0,NaN
5583,5988,"@USER: Stand By Your Ad should be applied to online ads, robo-calls. #MoneyInPolitics",0,1,0,0,1.0,0.0,0.0,0.0


In [17]:
#classify messages using gpt4o:

jaidkaGPTrun4 = ['rationality_jaidka']
jaidkaGPTrun5 = ['interactivity_acknowledgement_simple', 'incivility_simple2', 'constructiveness_jaidka', 'political_post_jaidka', 'rationality_simple2']
jaidkaGPTrun6 = ['civility_jaidka', 'incivility_jaidka']
jaidkaGPTrun7 = ['interactivity_acknowledgement_jaidka']
jaidkaGPTrun8 = ['political_post']
  

chunked_result: typing.List[pd.DataFrame] = []

for label, path in CFG.prompt_classify_files.items():
    if label in jaidkaGPTrun8: 
        template = load_json(path).get('template')
        classes = load_json(path).get('classes')
        for index, row in tqdm.tqdm(data_jaidka["message"].items()):
            retry_count = 0
            max_retries = 10
            while retry_count < max_retries:
                try: 
                    response = requests.post(
                            url=api_endpoint,
                            headers=headers,
                            json={
                                'model': MODELgpt4o,
                                'messages': [
                                    {
                                        "role": "system",
                                        "content": template
                                    },
                                    {
                                        "role": "user",
                                        "content": template.format(text=row)
                                    }
                                ],
                                'temperature': temperature_0,  
                                'seed': SEED,
                                "max_tokens": MAX10
                            }
                        )  

                    if response.status_code == 200:
                        data_response = response.json()
                        chunked_result.append(
                        pd.DataFrame(
                            data=[[index, classes.get(data_response["choices"][0]["message"]["content"], None)]],                                
                            columns=['index', label]
                            )
                        )
                        break  # Exit the retry loop on success
                    elif response.status_code == 429:
                        retry_count += 1
                        wait_time = 1 + (3 * retry_count * retry_count)
                        print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                        print(response.text)
                        time.sleep(wait_time)
                    elif response.status_code == 500:
                        retry_count += 1
                        wait_time = 20
                        print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                        print(response.text)
                        time.sleep(wait_time)
                    else:
                        print(f"Failed to connect to API. Status code: {response.status_code}")
                        print(response.text)
                        break
                except requests.exceptions.RequestException as e:   
                    print(f"Failed to connect to API: {e}")
                    retry_count += 1
                    wait_time = 60
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)                 
                
            #Add a pause of 1 minute after every 70 iterations
            #if (index + 1) % 250 == 0:
            #    print("Pausing for 10 seconds...")
            #    time.sleep(10)

classifications4 = pd.concat(chunked_result, ignore_index=True)
print(classifications4) 

5585it [1:38:44,  1.06s/it]

      index political_post
0         0      political
1         1      political
2         2  non-political
3         3      political
4         4  non-political
...     ...            ...
5580   5580      political
5581   5581      political
5582   5582  non-political
5583   5583      political
5584   5584      political

[5585 rows x 2 columns]


In [ ]:
#read data:
data_jaidka2 = pd.read_parquet('data/jaidka2022/TwitterDeliberativePolitics.parquet')

In [ ]:
interactivity_acknowledgement_jaidka_gpt4o = classifications2.loc[:, ["index", "interactivity_acknowledgement_jaidka"]].dropna()
interactivity_acknowledgement_jaidka_gpt4o.set_index('index', drop=True, inplace=True)
data_jaidka2 = data_jaidka2.join(interactivity_acknowledgement_jaidka_gpt4o, rsuffix='_gpt4o')

In [77]:
political_post_jaidka_gpt4o = classifications3.loc[:, ["index", "political_post_jaidka"]].dropna()
political_post_jaidka_gpt4o.set_index('index', drop=True, inplace=True)
data_jaidka2 = data_jaidka2.join(political_post_jaidka_gpt4o, rsuffix='_gpt4o')

In [78]:
constructiveness_jaidka_gpt4o = classifications3.loc[:, ["index", "constructiveness_jaidka"]].dropna()
constructiveness_jaidka_gpt4o.set_index('index', drop=True, inplace=True)
data_jaidka2 = data_jaidka2.join(constructiveness_jaidka_gpt4o, rsuffix='_gpt4o')

In [79]:
interactivity_acknowledgement_simple_gpt4o = classifications3.loc[:, ["index", "interactivity_acknowledgement_simple"]].dropna()
interactivity_acknowledgement_simple_gpt4o.set_index('index', drop=True, inplace=True)
data_jaidka2 = data_jaidka2.join(interactivity_acknowledgement_simple_gpt4o, rsuffix='_gpt4o')

In [80]:
incivility_simple2_gpt4o = classifications3.loc[:, ["index", "incivility_simple2"]].dropna()
incivility_simple2_gpt4o.set_index('index', drop=True, inplace=True)
data_jaidka2 = data_jaidka2.join(incivility_simple2_gpt4o, rsuffix='_gpt4o')

In [18]:
political_post_gpt4o = classifications4.loc[:, ["index", "political_post"]].dropna()
political_post_gpt4o.set_index('index', drop=True, inplace=True)
data_jaidka2 = data_jaidka2.join(political_post_gpt4o, rsuffix='_gpt4o')

In [19]:
data_jaidka2 = data_jaidka2.rename(columns={'political_post_jaidka': 'political_post_jaidka_gpt4o', 'constructiveness_jaidka': 'constructiveness_jaidka_gpt4o', 'interactivity_acknowledgement_jaidka': 'interactivity_acknowledgement_jaidka_gpt4o', 'interactivity_acknowledgement_simple': 'interactivity_acknowledgement_simple_gpt4o', 'incivility_simple2': 'incivility_simple2_gpt4o', 'political_post': 'political_post_gpt4o'})

In [62]:
civility_gpt4o = classifications1.loc[:, ["index", "civility_jaidka"]].dropna()
incivility_gpt4o = classifications1.loc[:, ["index", "incivility_jaidka"]].dropna()

In [63]:
incivility_gpt4o.set_index('index', drop=True, inplace=True)
civility_gpt4o.set_index('index', drop=True, inplace=True)

In [7]:
#read data:
data_jaidka2 = pd.read_parquet('data/jaidka2022/TwitterDeliberativePolitics2.parquet')

In [28]:
data_jaidka2.loc[:, ['reciprocity_jaidka_gpt4o', 'civility_jaidka_gpt4o',
       'incivility_jaidka_gpt4o', 'political_post_jaidka_gpt4o',
       'constructiveness_jaidka_gpt4o',
       'interactivity_acknowledgement_simple_gpt4o',
       'incivility_simple2_gpt4o', "political_post_gpt4o_dum"]]

,reciprocity_jaidka_gpt4o,civility_jaidka_gpt4o,incivility_jaidka_gpt4o,political_post_jaidka_gpt4o,constructiveness_jaidka_gpt4o,interactivity_acknowledgement_simple_gpt4o,incivility_simple2_gpt4o,political_post_gpt4o_dum
0,No,No,No,Yes,Yes,Yes,No,0
1,No,No,Yes,Yes,No,Yes,Yes,0
2,No,Yes,No,No,Yes,Yes,No,0
3,No,No,No,Yes,No,Yes,No,0
4,No,No,Yes,No,No,No,Yes,0
...,...,...,...,...,...,...,...,...
5580,No,Yes,No,Yes,No,Yes,No,0
5581,No,No,No,Yes,No,Yes,No,0
5582,No,No,Yes,Yes,No,Yes,Yes,0
5583,No,Yes,No,Yes,Yes,Yes,No,0


In [83]:
#save data:
data_jaidka2.to_parquet('data/jaidka2022/TwitterDeliberativePolitics2.parquet')

In [59]:
data_jaidka2 = data_jaidka2.join(interactivity_acknowledgement_jaidka_gpt4o, rsuffix='_gpt4o')
data_jaidka2 = data_jaidka2.rename(columns={'interactivity_acknowledgement_jaidka': 'reciprocity_jaidka_gpt4o'})
data_jaidka2 = data_jaidka2.join(civility_gpt4o, rsuffix='_gpt4o').join(incivility_gpt4o, rsuffix='_gpt4o')
data_jaidka2 = data_jaidka2.rename(columns={'civility_jaidka': 'civility_jaidka_gpt4o', 'incivility_jaidka': 'incivility_jaidka_gpt4o'})


In [69]:
classifications1.set_index('index', drop=True, inplace=True)
data_jaidka = data_jaidka.join(classifications1, rsuffix='_gpt4o')

In [31]:
data_jaidka2.loc[:, 'rationality_jaidka_gpt4o_dum'] = data_jaidka2.loc[:, 'rationality_jaidka_gpt4o'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
data_jaidka2.loc[:, 'civility_jaidka_gpt4o_dum'] = data_jaidka2.loc[:, 'civility_jaidka_gpt4o'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
data_jaidka2.loc[:, 'incivility_jaidka_gpt4o_dum'] = data_jaidka2.loc[:, 'incivility_jaidka_gpt4o'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
data_jaidka2.loc[:, 'reciprocity_jaidka_gpt4o_dum'] = data_jaidka2.loc[:, 'reciprocity_jaidka_gpt4o'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
data_jaidka2.loc[:, 'political_post_jaidka_gpt4o_dum'] = data_jaidka2.loc[:, 'political_post_jaidka_gpt4o'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
data_jaidka2.loc[:, 'constructiveness_jaidka_gpt4o_dum'] = data_jaidka2.loc[:, 'constructiveness_jaidka_gpt4o'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
data_jaidka2.loc[:, 'interactivity_acknowledgement_simple_gpt4o_dum'] = data_jaidka2.loc[:, 'interactivity_acknowledgement_simple_gpt4o'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
data_jaidka2.loc[:, 'incivility_simple2_gpt4o_dum'] = data_jaidka2.loc[:, 'incivility_simple2_gpt4o'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
data_jaidka2.loc[:, 'political_post_gpt4o_dum'] = data_jaidka2.loc[:, 'political_post_gpt4o'].map({"political": 1, "non-political":0}).fillna(0).astype(int)

In [22]:
data_jaidka2.columns

Index(['message_id', 'message', 'Constructiveness', 'Justification',
       'Justification_internal', 'Justification_external', 'Relevance',
       'Reciprocity', 'Empathy_Respect', 'Uncivil_abuse',
       'rationality_simple2_Llama8b', 'rationality_simple2_Llama8b_dum',
       'rationality_jaidka_Llama8b', 'rationality_jaidka_Llama8b_dum',
       'rationality_jaidka_gpt4o', 'rationality_jaidka_gpt4o_dum',
       'reciprocity_jaidka_gpt4o', 'civility_jaidka_gpt4o',
       'incivility_jaidka_gpt4o', 'political_post_jaidka_gpt4o',
       'constructiveness_jaidka_gpt4o',
       'interactivity_acknowledgement_simple_gpt4o',
       'incivility_simple2_gpt4o', 'civility_jaidka_gpt4o_dum',
       'incivility_jaidka_gpt4o_dum', 'reciprocity_jaidka_gpt4o_dum',
       'political_post_jaidka_gpt4o_dum', 'constructiveness_jaidka_gpt4o_dum',
       'interactivity_acknowledgement_simple_gpt4o_dum',
       'incivility_simple2_gpt4o_dum', 'political_post_gpt4o',
       'political_post_gpt4o_dum'],
   

In [34]:
#save data:
data_jaidka2.to_parquet('data/jaidka2022/TwitterDeliberativePolitics2.parquet')

In [19]:
#data_jaidka2 = data_jaidka2.join(classifications1, rsuffix='_gpt4o')
#data_jaidka2 = data_jaidka2.rename(columns={'civility_jaidka': 'civility_jaidka_gpt4o', 'incivility_jaidka': 'incivility_jaidka_gpt4o'})

In [35]:
#evaluate performance of gpt4o on the Jaidka set:
llm_human_column_pairs = [
    ("Justification", "rationality_jaidka_gpt4o_dum"),
    ("Empathy_Respect", 'civility_jaidka_gpt4o_dum'),
    ('Empathy_Respect','incivility_simple2_gpt4o_dum'), 
    ('Uncivil_abuse', 'incivility_jaidka_gpt4o_dum'),
    ('Uncivil_abuse','incivility_simple2_gpt4o_dum'), 
    ("Reciprocity", 'reciprocity_jaidka_gpt4o_dum'), 
    ("Reciprocity",'interactivity_acknowledgement_simple_gpt4o_dum'),
    ("Relevance", 'political_post_jaidka_gpt4o_dum'),
    ("Relevance", 'political_post_gpt4o_dum'), 
    ('Constructiveness', 'constructiveness_jaidka_gpt4o_dum')
   ]
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = data_jaidka2.loc[:, [human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

    # Calculate percent agreement
    agreement = np.sum(np.array(human) == np.array(llm))/len(np.array(human)) * 100

    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")
    print(f"percent_agreement({human_col}): {agreement:.2f}%")


Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.42      0.82      0.56      2005
           1       0.79      0.38      0.51      3580

    accuracy                           0.54      5585
   macro avg       0.60      0.60      0.53      5585
weighted avg       0.66      0.54      0.53      5585

cohen_kappa_score(Justification): 0.16090861739684004
krippendorf(Justification): 0.06809121665923579
percent_agreement(Justification): 53.52%
---
              precision    recall  f1-score   support

         0.0       0.39      0.97      0.56      1465
         1.0       0.94      0.20      0.33      2813

    accuracy                           0.47      4278
   macro avg       0.66      0.59      0.44      4278
weighted avg       0.75      0.47      0.41      4278

cohen_kappa_score(Empathy_Respect): 0.12937442176914427
krippendorf(Empathy_Respect): -0.1

In [ ]:
#what would the Llama3.1_8b model do?

jaidkaLlamarun1 = ['rationality_simple2']
jaidkaLlamarun2 = ['rationality_jaidka']

jaidkaLlamarun3 = ['interactivity_acknowledgement_simple', 'incivility_simple2']
#pubspherepromptsrun25 = ['interactivity_acknowledgement_simple', 'political_ideology_US', 'incivility_simple2', 'rationality_simple2']   

predictions2L: typing.Dict[str, np.ndarray] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (data_jaidka["message"], model=MODELsmall, options=options_zero)
    )
    for label, path in CFG.prompt_classify_files.items() if label in jaidkaLlamarun2
}

classifying rationality_jaidka: 100%|██████████| 5585/5585 [27:07<00:00,  3.43it/s] 


In [ ]:
#what would the Llama3.1_70b model do?

jaidkaLlamarun1 = ['rationality_simple2']
jaidkaLlamarun2 = ['rationality_jaidka']

jaidkaLlamarun3 = ['interactivity_acknowledgement_simple', 'incivility_simple2']
#pubspherepromptsrun25 = ['interactivity_acknowledgement_simple', 'political_ideology_US', 'incivility_simple2', 'rationality_simple2']   

predictions2_L70: typing.Dict[str, np.ndarray] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (data_jaidka["message"], model=MODELlarge, options=options_zero)
    )
    for label, path in CFG.prompt_classify_files.items() if label in jaidkaLlamarun2
}

In [7]:
#join to the dataset:   
for _, preds in predictions1L.items():
    print(preds.value_counts())
    print("-" * 42)
        # Rename the column in preds
    preds = preds.rename('rationality_simple2_Llama8b')
    
    data_jaidka = data_jaidka.join(preds, rsuffix='_Llama8b')


rationality_simple2
No     5505
Yes      54
Name: count, dtype: int64
------------------------------------------


In [15]:
#join to the dataset:   
for _, preds in predictions2L.items():
    print(preds.value_counts())
    print("-" * 42)
        # Rename the column in preds
    preds = preds.rename('rationality_jaidka_Llama8b')
    
    data_jaidka = data_jaidka.join(preds, rsuffix='_Llama8b')

rationality_jaidka
Yes    4664
No      410
Name: count, dtype: int64
------------------------------------------


In [16]:
data_jaidka.loc[:, 'rationality_jaidka_Llama8b_dum'] = data_jaidka.loc[:, 'rationality_jaidka_Llama8b'].map({"Yes": 1, "No":0}).fillna(0).astype(int)


In [ ]:
data_jaidka.loc[:, 'incivility_simple2_Llama8b_dum'] = data_jaidka.loc[:, 'incivility_simple2_Llama8b'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
data_jaidka.loc[:, 'rationality_simple2_Llama8b_dum'] = data_jaidka.loc[:, 'rationality_simple2_Llama8b'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
data_jaidka.loc[:, 'interactivity_acknowledgement_simple_Llama8b_dum'] = data_jaidka.loc[:, 'interactivity_acknowledgement_simple_Llama8b'].map({"Yes": 1, "No":0}).fillna(0).astype(int)

data_jaidka.loc[:, 'Incivility_tot'] = data_jaidka.loc[:, ['Uncivil_abuse', 'Empathy_Respect']].max(axis=1)


In [17]:
#evaluate performance of Llama3.1_8b model on the Jaidka set:
llm_human_column_pairs = [
    ("Justification", "rationality_simple2_Llama8b_dum"),
    ("Justification", "rationality_jaidka_Llama8b_dum"),
#    ("Incivility_tot", "incivility_simple2_Llama8b_dum"),
#    ("Uncivil_abuse", "incivility_simple2_Llama8b_dum"),
#    ("Reciprocity", "interactivity_acknowledgement_simple_Llama8b_dum")
]
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = data_jaidka.loc[:, [human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

    # Calculate percent agreement
    agreement = np.sum(np.array(human) == np.array(llm))/len(np.array(human)) * 100

    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")
    print(f"percent_agreement({human_col}): {agreement:.2f}%")


Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.36      1.00      0.53      2005
           1       0.83      0.01      0.02      3580

    accuracy                           0.37      5585
   macro avg       0.60      0.50      0.28      5585
weighted avg       0.66      0.37      0.21      5585

cohen_kappa_score(Justification): 0.005826939739066006
krippendorf(Justification): -0.44538050051004485
percent_agreement(Justification): 36.54%
---
              precision    recall  f1-score   support

           0       0.49      0.23      0.31      2005
           1       0.67      0.87      0.75      3580

    accuracy                           0.64      5585
   macro avg       0.58      0.55      0.53      5585
weighted avg       0.60      0.64      0.60      5585

cohen_kappa_score(Justification): 0.10806024429774286
krippendorf(Justification): 0.0646

In [18]:
#the pubsphere prompt is strongly inferior to SML reported in the paper with minority F1 0.54 and macro F1 0.71....
#the Jaidka prompt is better, but still not as good as the SML reported in the paper


In [19]:
#save data:
data_jaidka.to_parquet('data/jaidka2022/TwitterDeliberativePolitics.parquet')

In [3]:
#read data:
data_jaidka = pd.read_parquet('data/jaidka2022/TwitterDeliberativePolitics.parquet')

In [5]:
#evaluate performance of gpt4o on the Jaidka set:
llm_human_column_pairs = [
    ("Justification", "rationality_jaidka_gpt4o_dum"),
    ("Justification", "rationality_jaidka_Llama8b_dum"),
   ]
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = data_jaidka.loc[:, [human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

    # Calculate percent agreement
    agreement = np.sum(np.array(human) == np.array(llm))/len(np.array(human)) * 100

    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")
    print(f"percent_agreement({human_col}): {agreement:.2f}%")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.42      0.82      0.56      2005
           1       0.79      0.38      0.51      3580

    accuracy                           0.54      5585
   macro avg       0.60      0.60      0.53      5585
weighted avg       0.66      0.54      0.53      5585

cohen_kappa_score(Justification): 0.16090861739684004
krippendorf(Justification): 0.06809121665923579
percent_agreement(Justification): 53.52%
---
              precision    recall  f1-score   support

           0       0.49      0.23      0.31      2005
           1       0.67      0.87      0.75      3580

    accuracy                           0.64      5585
   macro avg       0.58      0.55      0.53      5585
weighted avg       0.60      0.64      0.60      5585

cohen_kappa_score(Justification): 0.10806024429774286
krippendorf(Justification): 0.064694

In [ ]:
#very similar to Llama and inferior to the SML reported in the paper, however much better balanced between classes than the Llama model

In [4]:
#however, note that Jaike disregarded missing manual annotated labels, so the performance might be underreported here
#df = df[df[y_col].notna()] in Github\deliberative-politics\notebooks\02 - Deliberation - baselines.ipynb
#check whether we have done the same, we used the .dropna() method, so should be similar:

#load data:
data_jaidka2 = pd.read_parquet('data/jaidka2022/TwitterDeliberativePolitics2.parquet')
data_jaidka2.loc[:, ['Justification', 'rationality_jaidka_gpt4o_dum']].dropna().value_counts()

Justification  rationality_jaidka_gpt4o_dum
1              0                               2226
0              0                               1635
1              1                               1354
0              1                                370
Name: count, dtype: int64

indeed same as Jaika...

In [ ]:
#how about the other variables?
